In [1]:
%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby

import distance
import nltk
#nltk.download('stopwords')

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

In [2]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [3]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [4]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [5]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [6]:
def stimPhrasesEdit_expt2019Dec(PhraseLog, subjName):
    
    # Now extract phrases from the phrase file
    phraseStim_Phrases = [item[1] for item in PhraseLog]
    phraseStim_PhrasesReduced = sorted(set(phraseStim_Phrases), key=phraseStim_Phrases.index)
    
    PhraseLogReduced= list()
    ind = -1
    
    #print(phraseStim_PhrasesReduced)
    
    for i in phraseStim_PhrasesReduced:
        ind = ind + 1

        if ind == len(phraseStim_PhrasesReduced)-1:
            if subjName == 'sa\\Test_woChinRest\\p1':
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i[0:31]])
                
            elif subjName == 'rh\\Test_wChinRest\\p1':
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i[0:-9]])
            else:
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i])
        else:
            PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i])
            
    del PhraseLogReduced[0]
    del PhraseLogReduced[0]
    

    
    if PhraseLogReduced[-1][1] == 'THE EXPERIMENT IS NOW DONE':
        del PhraseLogReduced[-1]
        
    if subjName == 'rh\\Test_wChinRest\\p2':
        #print(PhraseLogReduced[2])
        del PhraseLogReduced[2]
        #print(PhraseLogReduced[-1])
        del PhraseLogReduced[-1]
        
    #if subjName == 'sa\\Test_woChinRest\\p1' or subjName == 'rh\\Test_wChinRest\\p1':
    #    del PhraseLogReduced[-1]
        
    if subjName == 'sa\\Test_woChinRest\\p2':
        del PhraseLogReduced[3]
        del PhraseLogReduced[1]
        del PhraseLogReduced[0]
        
    return PhraseLogReduced
    
    

In [7]:
def scratchPadPhraseEdit_expt2019Dec(phraseUser, subjName):
    phraseUserEnd = list()
    
    print(subjName)
    
    #print(phraseUser)
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            if len(phraseUser[row_ind-1][1])>10:
                phraseUserEnd.append(phraseUser[row_ind-1])
                #print(phraseUser[row_ind-1])
    
    if subjName == 'sa\\Test_woChinRest\\p1':
        phraseUserEnd.append([phraseUser[-1][0], phraseUser[-1][1][0:35]])
    
    if subjName == 'rh\\Test_wChinRest\\p1':
        phraseUserEnd.append(phraseUser[-1])
        
    # remove first trial of text composition
    del phraseUserEnd[0]
        
    if subjName == 'rh\\Test_wChinRest\\p2':
        print('deleting: ', phraseUserEnd[-2])
        del phraseUserEnd[-2]
        
    return phraseUserEnd

In [8]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [38]:
def rindex(mylist, myvalue):
    return len(mylist) - mylist[::-1].index(myvalue) - 1

In [130]:
def LettersNotSelected(UserKeys, PhrasesStim):
    
    UserKeysTime = [val[0] for val in UserKeys]
    
    lettersNotSelected = list()
    
    # find starting point of every phrase 
    for ind in range(0,len(PhrasesStim)):
        #print(' ')
        #print('phrase : ', PhrasesStim[ind][1])
        timePhraseStarts = PhrasesStim[ind][0]
        
        # Find the time when the phrase starts
        timePhraseStartsWUserKeys, ind_start = nearestTimePoint(UserKeysTime, timePhraseStarts)
        
        
        # find when the phrase ends:
        if ind == len(PhrasesStim)-1: # if it is the last sentence, reverse index searching starts from the last index
            ind_end = len(UserKeys) - 1
        else: # if it's not the last sentence, reverse index searching starts from the next phrase
            ind_end_time, ind_end = nearestTimePoint(UserKeysTime, PhrasesStim[ind+1][0])
         
        '''''
        
        print(ind_end1, UserKeys[ind_end1])
        
        # reverse indexing to find 'NextPhrase'
        ind_end2 = ind_end1
        while ind_end2 > ind_start+1:
            ind_end2 = ind_end2 - 1
            if UserKeys[ind_end2][1] == 'NextPhrase' and UserKeys[ind_end1][2]==1:
                ind_end_start = ind_end2
        
        if ind_end_start is None:
            ind_end_start = ind_end1
        
        while ind_end_start > ind_start + 1:
            ind_end_start = ind_end_start - 1
            if UserKeys[ind_end_start][1] == 'NextPhrase' and UserKeys[ind_end_start-1][1] != 'NextPhrase':
                ind_end = ind_end_start
                break
                
        if ind_end is None:
            ind_end = ind_end1
             
        '''''
        
        flag_start = 0
        typing_list = list()
        letter_list = list()
        
        # establish dwell time for every selection
        dwellTime_current = 800 # assume for the beginning
        
        # collect the typing letters in a list of lists
        for ind_typing in range(ind_start, ind_end):
            
            if len(UserKeys[ind_typing][1]) == 1 or UserKeys[ind_typing][1] == 'BackOne' or UserKeys[ind_typing][1] == 'BackMany' or UserKeys[ind_typing][1] == 'LeftShift' or UserKeys[ind_typing][1] == 'SpaceBar' or UserKeys[ind_typing][1] == 'ScratchPad' or 'Suggestion' in UserKeys[ind_typing][1]: 
                #print(UserKeys[ind_typing][1])
                letter_list.append(UserKeys[ind_typing]) 
                if UserKeys[ind_typing][1] != UserKeys[ind_typing+1][1]:
                    #if len(UserKeys[ind_typing+1][1]) == 1 or UserKeys[ind_typing+1][1] == 'BackOne' or UserKeys[ind_typing+1][1] == 'BackMany' or UserKeys[ind_typing+1][1] == 'LeftShift' or UserKeys[ind_typing+1][1] == 'SpaceBar':
                    typing_list.append(letter_list)
                    letter_list = list()
            
        
        
        
        # from the collected letters, find the ones that are not selected
        typing_list_temp = list()
        typing_list_ind = list()
        startingProgressPct_list = list()
        
        ind_letter = -1
        for letter in typing_list:
            #print('ind: ', ind_letter)
            #print(' before 2nd for loop: ', typing_list_temp)
            ind_letter = ind_letter + 1
            #print(letter[0])
            
            typing_list_temp.append(letter[0][1])
            typing_list_ind.append(ind_letter)
            #print('Before, the list is: ', typing_list_temp, 'as', letter[0], 'added')
            
            #print('Added ', letter[0][1], 'to make the list: ----------------', typing_list_temp )
            #print('The index of the letter was ', typing_list_ind[-1])
            #print('the lengths of the lists are - ', len(typing_list_temp), len(typing_list_ind))
            
            start = 0
            for instance in letter:
                
                foundBeginning = 0
                
                if instance[1] == 'ScratchPad':
                    continue
                
                if start < 1:
                    startingProgressPct_list.append(float(instance[2]))
                    #print('starting progress', startingProgressPct_list[-1], ' at ', instance)
                    #print('complete list: ', startingProgressPct_list)
                    start = 1
                
                #print(instance)
                
                
                if int(instance[2]) == 1:
                    print(instance)
                    # update the dwell time
                    dwellTime_current = float(instance[3])
                    
                    
                        
                    # this means that it is not the complete one-time selection
                    if startingProgressPct_list[-1] > 0.02:
                        # start checking the progress, by finding the latest key with 0 progress that led to the current selection
                        endInd_progressCheck = -1
                        # check if it was a continuation of a previous fixation, and if the previous one also needs to be removed
                        while foundBeginning < 1:
                            print('endProgressInd', endInd_progressCheck)
                            #print(foundBeginning)
                            #print(startingProgressPct_list[endInd_progressCheck], endInd_progressCheck)
                            
                            #if len(typing_list_temp) > 2 and letter[0][1] == typing_list_temp[-3] and letter[0][2] >= typing_list[ind_letter-2][-1][2]:
                            if letter[0][1] in typing_list_temp[0:endInd_progressCheck]:
                                
                                ind_letter2 = rindex(typing_list_temp[0:endInd_progressCheck], letter[0][1]) # find the last index of the key
                                ind_letterCompl = typing_list_ind[ind_letter2]
                                endInd_progressCheck = ind_letter2
                                print(endInd_progressCheck, startingProgressPct_list[endInd_progressCheck], letter[0][1], typing_list_temp[endInd_progressCheck])
                                
                                #print('letter', typing_list_temp[ind_letter2], 'has the index', ind_letterCompl)
                                #print('just to be sure', typing_list_temp[])
                        
                                if float(letter[0][2]) >= float(typing_list[ind_letterCompl][-1][2]):
                                    if (float(letter[0][2]) - float(typing_list[ind_letterCompl][-1][2]))*dwellTime_current < 0.02*dwellTime_current:
                                        #print(letter[0][2], 'and', typing_list[ind_letterCompl][-1][2])
                                        #print('deleting because it turned into fixation: ', typing_list_temp[ind_letter2])
                                        del typing_list_temp[ind_letter2]
                                        del typing_list_ind[ind_letter2]
                                        
                       
                                # now check if we need to go back to find the starting point of the selection
                                #print('checking the progress pct', startingProgressPct_list[endInd_progressCheck], len(startingProgressPct_list), endInd_progressCheck)
                                if startingProgressPct_list[endInd_progressCheck] < 0.02:
                                    foundBeginning = 1
                                    print(startingProgressPct_list[endInd_progressCheck])
                                    print('foundBeginning')
                                    
                                else:
                                    #print('not found beginning')
                                    endInd_progressCheck = endInd_progressCheck - 1
                                    #print('new ind end', endInd_progressCheck)
                                    
                            else:
                                foundBeginning = 1
                        
                    #print('deleting: ', typing_list_temp[-1])
                    
                    del typing_list_temp[-1]
                    del typing_list_ind[-1]
                    
                    #print('And now the list is: ----------------', typing_list_temp)
                    
                        
                    break
        
        #print('And NOT selected: ', typing_list_temp)
        
        
        #ind = -1
        #for letter in typing_list_temp:
        #    ind = ind + 1
        #    print(ind, letter)
            
        
                    
        '''''
        # add the time for every letter in a phrase
        time_typing_letter = 0
        for letter_typed_list in typing_list:
            time1, t1, t2 = letter_typed_list[0].partition('+')
            timeStart = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            time2, t1, t2 = letter_typed_list[-1].partition('+')
            timeEnd = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            timeDiff = timeEnd - timeStart
            
            time_typing_letter = time_typing_letter + timeDiff.total_seconds()
        
    timeTypingList.append(time_typing_letter)
    '''''
        
        # count number of times looked at scratchpad during writing
        nTimes_ScratchPad = typing_list_temp.count('ScratchPad')
        print(nTimes_ScratchPad)
        
        nTrue_Suggestion = [True for item in typing_list_temp if 'Suggestion' in item] 
        nTimes_Suggestion = len(nTrue_Suggestion)
        print(nTimes_Suggestion)
        # remove scratchPad from the list:
        typing_list_woScratchPadOrSuggestion = [item for item in typing_list_temp if (item!= 'ScratchPad' and 'Suggestion' not in item)]
        
        
        #
        print(typing_list_woScratchPadOrSuggestion)
    
        #print(''.join(typing_list_temp))
        if len(typing_list_temp) > 1:
            #print(typing_list_temp)
            lettersNotSelected.append(typing_list_woScratchPadOrSuggestion)
            #print(len(typing_list_temp))
        
    
        
    return lettersNotSelected

In [134]:
metricComputed = 'typing_speed'
#dataFolderName = r'D:\Data'
dataFolderName = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data'

j = 0
TimeDwellAvg = 0
#nSub = 0
TypingSpeed = list()
Names = list()
ans_list = list()

for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        if 'rh' not in root:
            continue
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'ReplacementPhraseLog.csv'):
                
                try:
                    fPhraseLog = open(root + '\\' + file)
                    readerPhraseLog = csv.reader(fPhraseLog)
                    phraseLog = list(readerPhraseLog)
                        
                except:
                    if fPhraseLog is not None:
                        print('closing')
                        fPhraseLog.close()
                    else:
                        print('error in opening the replacement phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
                    
                     
        if scratchPad is None or userKeys is None or phraseLog is None:
            continue
        else:
            #a = re.compile('(?<=201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
            a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')
            subjName = a.findall(root)[0]
            print('subject id: ', subjName)
            print(root)
            
            # fix scratchpad due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
                
            # find dwell time of typing
            userKeys_wDwellTime = ComputeDwellTime(userKeys_new)
                
            phraseStim_reduced = stimPhrasesEdit_expt2019Dec(phraseLog_new, subjName)
            
            
            phraseUserEnd_reduced = scratchPadPhraseEdit_expt2019Dec(scratchPad_new, subjName)
                
                                
            # check if the number of sentences match in the input and the output
            #if len(phraseUserEnd_reduced)!=len(phraseStim_reduced):
            #    phraseStim_toMatch, phraseUserEnd_toMatch = MatchPhraseNumbers(phraseStim_reduced, phraseUserEnd_reduced)
            
            # Find the number of correct letters typed
            #ComputeCorrectLetters(phraseStim_reduced, phraseUserEnd_reduced)
            
            
            # Find letters not selected:
            # Find all keys that are not selected, just shortly looked at 
            letters_not_selected = LettersNotSelected(userKeys_wDwellTime, phraseStim_reduced)
            
            
            if 'sa\\Test_woChinRest' in subjName or 'rh\\Test_wChinRest' in subjName:
                    if 'p1' in root:
                        
                        if 'rh' in subjName:
                            letters_not_selected1 = list()
                            letters_not_selected1.append(letters_not_selected[0])
                            phraseUserEnd_reduced1 = phraseUserEnd_reduced[0]
                        else:
                            letters_not_selected1 = list()
                            letters_not_selected1 = letters_not_selected[0:-1]
                            #print('len', len(letters_not_selected1))
                            phraseUserEnd_reduced1 = phraseUserEnd_reduced[0:-1]
                        
                    else:
                        letters_not_selected2 = letters_not_selected
                        phraseUserEnd_reduced2 = phraseUserEnd_reduced
                   
            if subjName == 'sa\\Test_woChinRest\\p2' or subjName == 'rh\\Test_wChinRest\\p2':
                #print('Addition of lists')
                #print(phraseUserEnd_reduced1)
                letters_not_selected = letters_not_selected1 + letters_not_selected2   
                #print(len(letters_not_selected1), len(letters_not_selected2), len(letters_not_selected)) 
                phraseUserEnd_reduced = phraseUserEnd_reduced1 + phraseUserEnd_reduced2
            
            if 'p1' in subjName:
                continue
                
            #print('number of phrases', len(letters_not_selected))
            
            n_ind = -1
            for sentence_typed in phraseUserEnd_reduced:
                n_ind = n_ind + 1
                print(sentence_typed[1])
                print('letters not selected and sentence typed length: ', len(letters_not_selected[n_ind]), len(sentence_typed[1]))
                ans_list.append(len(letters_not_selected[n_ind])/len(sentence_typed[1]))
                
                
                
                
                

subject id:  rh\Test_wChinRest\p1
C:\DTU\Data\201812_ExptToCheckMovementEffect\Data\rh\Test_wChinRest\p1\2018-11-29-12-52-2
rh\Test_wChinRest\p1
['2018-11-29T13:00:37.8518106+01:00', 'h', 1.0, '800.0']
['2018-11-29T13:00:39.2515842+01:00', 'e', 1.0, '800.0']
endProgressInd -1
0 0.01371375 e e
0.01371375
foundBeginning
['2018-11-29T13:00:40.6398837+01:00', 'r', 1.0, '800.0']
endProgressInd -1
1 0.01371375 r r
0.01371375
foundBeginning
['2018-11-29T13:00:42.0501273+01:00', 'm', 1.0, '800.0']
endProgressInd -1
2 0.0 m m
0.0
foundBeginning
['2018-11-29T13:00:43.5042599+01:00', 'e', 1.0, '800.0']
['2018-11-29T13:00:44.9364442+01:00', 'd', 1.0, '800.0']
['2018-11-29T13:00:47.5903828+01:00', 'SpaceBar', 1.0, '800.0']
['2018-11-29T13:00:48.9777058+01:00', 'm', 1.0, '800.0']
endProgressInd -1
6 0.62457975 m m
endProgressInd 5
['2018-11-29T13:00:50.8546960+01:00', 'o', 1.0, '800.0']
endProgressInd -1
9 0.625212625 o o
endProgressInd 8
7 0.013713375 o o
0.013713375
foundBeginning
['2018-11-29T13:

IndexError: list index out of range

In [133]:
a = [3, 4, 4, 5, 4]
len(a) - a[::-1].index(4) - 1

4

In [21]:
print(ans_list)

[0.6610169491525424, 1.0, 1.1204819277108433, 0.48148148148148145, 1.273972602739726, 1.2149532710280373, 1.671641791044776, 0.6470588235294118, 0.7894736842105263, 0.41935483870967744, 1.9174311926605505, 3.4150943396226414, 2.5833333333333335, 1.8085106382978724, 1.7818181818181817, 1.421487603305785, 2.3333333333333335]


In [ ]:
# check using character levenshtein distance
                dissimilarity_check_2ndLastStimPhrase = distance.levenshtein(phraseLog_reduced[-2][1].lower(), phraseUserEnd_reduced[-1][1].lower())
                dissimilarity_check_LastStimPhrase = distance.levenshtein(phraseLog_reduced[-1][1].lower(), phraseUserEnd_reduced[-1][1].lower())
                
                print(dissimilarity_check_LastStimPhrase, dissimilarity_check_2ndLastStimPhrase)
                if dissimilarity_check_LastStimPhrase > dissimilarity_check_2ndLastStimPhrase:
                    del phraseLog_reduced[-1]

In [ ]:
The large scale orchestral introduction made use of the well known technique
Instead the organisation may blame its failures on the person it failed,  even unfairly damaging the person's reputation

In [ ]:
def phrasesTyped(your_list, timeList, userKeys, stimPhraseList):
    # Find entries in scratchpad (end of each trial)
    
    IndexNotInclude = list()
    PhraseList = list()
    TypingSpeed = list()

    # After every complete phrase, the next phrase is empyty ''
    for row_ind in range(0, len(your_list)):
        if your_list[row_ind][1] == '':
            if row_ind == 1:# first empty phrase needs to be not considered , at least for most of the subjects
                continue
            if your_list[row_ind][1] == your_list[row_ind-1][1]:
                continue
            IndexNotInclude.append(row_ind)
    
    # Find the time taken for each phrase  

    #print(len(timeList), len(stimPhraseList), len(IndexNotInclude))
    #print(stimPhraseList)
    if len(IndexNotInclude) != len(timeList) :
        #print(IndexNotInclude, timeList)
        print('error and both are not same')
        #print(IndexNotInclude)
        #print(scratchPad)
    else:
        
        for indPhrase in range(0, len(IndexNotInclude)):
            startPhrase = timeList[indPhrase]    
            endPhrase = IndexNotInclude[indPhrase]-1
            headStartPhrase,sep,tail = userKeys[startPhrase][0].partition('+')
            headEndPhrase,sep,tail = your_list[endPhrase][0].partition('+')
            timeTaken = datetime.datetime.strptime(re.sub('[:.T]','-',headEndPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f") - datetime.datetime.strptime(re.sub('[:.T]', '-', headStartPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            phraseTyped = your_list[endPhrase][1]
            PhraseList.append([timeTaken, phraseTyped])
            #print(startPhrase, ',', endPhrase, ',', headStartPhrase, ',', headEndPhrase, ',', timeTaken, ',', phraseTyped)
            
        
    #print(timeTaken)
#    print(PhraseList)
    
    # Compute the total time for the phrase and the total number of words typed during that time (1 word = 5 characters, including space)
    timeAdd = datetime.datetime.strptime('1000-1-1-0-00-00-000000',"%Y-%m-%d-%H-%M-%S-%f")
    characterLen = 0
    
    for indPhrase in range(0, len(IndexNotInclude)):
        timeAdd += PhraseList[indPhrase][0]
        characterLen += len(PhraseList[indPhrase][1])
#        print(timeAdd, ': ', characterLen)
#    print(timeAdd, ': ', characterLen)
        
    TimeTaken = (timeAdd.microsecond/1000000 + timeAdd.second)/60 + timeAdd.minute
#    print(TimeTaken)
    WPM = (characterLen)/(5*TimeTaken)
    print("Typing speed is", WPM)
    
    return WPM

In [ ]:
def AddDataToFile(metricComputed, Names, DataEvaluated):

    # Save the data
    fileIn = r'C:\DTU\Data\201805_HealthnRehab\data_summary.csv'

    if Path(fileIn).is_file():
        print(fileIn)
        fSize = os.path.getsize(fileIn) # or pd.read_excel(filename) for xls file

        # file exists
        # check if the given column exists
        with open(fileIn, 'r', newline='') as csvfileRead:
            if fSize > 0:
                fileOut = fileIn[:-4] + '1.csv'
                with open(fileOut, 'w', newline='') as csvfileWrite:
                    for row in csv.reader(csvfileRead):
                        if metricComputed in row:
                            # file exists and the metric is present in the file
                            print('This metric is already calculated and saved: ', metricComputed)
                            break
                        else:
                            # file exists but the metric is not present
                            print('adding the metric')
                            filewriter = csv.writer(csvfileWrite, delimiter = ',')
                            if len(row) > 1:
                                rowJoined = list()
                                for rowElement in row:
                                    rowJoined.append(rowElement)
                                
                                
                                print(rowJoined[0])
                                
                                if rowJoined[1] == 'subject_name':
                                    # write the titles of the metrics
                                    rowJoined.append(metricComputed)
                                    filewriter.writerow(rowJoined)
                                else:
                                    #filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
                                    rowJoined.append(str(DataEvaluated[Names.index(row[1])]))
                                    filewriter.writerow(rowJoined)
                                
                            else:
                                if row[1] == 'subject_name':
                                    # write the titles of the metrics
                                    filewriter.writerow([row, metricComputed])
                                else:
                                    filewriter.writerow([row, str(DataEvaluated[Names.index(row[1])])])
            
            
            else:
                fileOut = fileIn
                print('recreating a file')

                with open(fileSave, 'w', newline='') as csvfileWrite:
                    filewriter = csv.writer(csvfileWrite, delimiter=',')
                    filewriter.writerow(['subject_name', metricComputed])
                    for ind in enumerate(Names):
                        filewriter.writerow([Names[ind], DataEvaluated[ind[0]]])
                
    else:
        print('creating a file')
        a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
        with open(fileSave, 'w', newline='') as csvfileWrite:
            filewriter = csv.writer(csvfileWrite, delimiter=',')
            filewriter.writerow(['subject_name', metricComputed])
            for ind in enumerate(Names):
                subjName = a.findall(Names[ind[0]])
                filewriter.writerow([subjName, DataEvaluated[ind[0]]])
    

In [ ]:
def phrasesTyped2(your_list):
    # Find entries in scratchpad that are the start and end of the phrase
    
    IndexNotInclude = list()
    
    # After every complete phrase, the next phrase is empyty '', and the first phrase is 'scratchPadText'
    for row_ind in range(0, len(your_list)):
        if your_list[row_ind][1] in NotInclude:
            if row_ind == 0:
                IndexNotInclude.append(row_ind) # first element is to be considered for the first phrase start time
                continue
            if your_list[row_ind][1] == your_list[row_ind-1][1]:
                continue
            IndexNotInclude.append(row_ind)
    
    for row_ind in range(0, len(your_list)):
        list_inner = your_list[row_ind]
        if list_inner[1] in NotInclude:
            IndexNotInclude.append(row_ind)
    
    # Remove consecutive entries with blank scratchpads

    IndexNotIncludeArray = np.asarray(IndexNotInclude)
    IndDiff = np.diff(IndexNotIncludeArray,1)
    
    ind = len(IndexNotInclude)-1
    while (ind>-1):
        if IndDiff[ind] == 1:
            for i_2 in range (ind,len(IndexNotInclude)):
                IndexNotInclude[i_2] = IndexNotInclude[i_2]-1
        
            del(IndexNotInclude[ind])
            del your_list[IndexNotIncludeArray[ind]]
        ind = ind - 1
        
        # Find the time taken for each phrase
    PhraseList = list()

    for indPhrase in range(0, len(IndexNotInclude)-1):
        startPhrase = IndexNotInclude[indPhrase]
        endPhrase = IndexNotInclude[indPhrase+1]
        headStartPhrase,sep,tail = your_list[startPhrase+1][0].partition('+')
        headEndPhrase,sep,tail = your_list[endPhrase-1][0].partition('+')
        print(headStartPhrase, headEndPhrase)
        timeTaken = datetime.datetime.strptime(re.sub('[:.T]','-',headEndPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f") - datetime.datetime.strptime(re.sub('[:.T]', '-', headStartPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
        phraseTyped = your_list[endPhrase-1][1]
        PhraseList.append([timeTaken, phraseTyped])
        
    #print(timeTaken)
    #print(PhraseList)
    
    # Compute the total time for the phrase and the total number of words typed during that time (1 word = 5 characters, including space)
    timeAdd = datetime.datetime.strptime('1000-1-1-0-00-00-000000',"%Y-%m-%d-%H-%M-%S-%f")
    characterLen = 0

    for indPhrase in range(0, len(IndexNotInclude)-1):
        timeAdd += PhraseList[indPhrase][0]
        characterLen += len(PhraseList[indPhrase][1])
    
    TimeTaken = (timeAdd.microsecond/1000000 + timeAdd.second)/60 + timeAdd.minute
    WPM = (characterLen)/(5*TimeTaken)
    #print("Typing speed is", WPM)